In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import psycopg2
from dotenv import load_dotenv
import os
import time

# Web Scrape Stats

In [13]:
def get_player_stats(first,last):
    """
    Returns stats and height of player obtained from sportsreference.com.
    """
    url = f'https://www.sports-reference.com/cbb/players/{first.lower()}-{last.lower()}-1.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract basic player info (height, weight, birth date)
    player_info = soup.find_all('p')
    height = None
    for info in player_info:
        if 'cm' in info.text and 'kg' in info.text:  # Height and weight are usually mentioned with these units
            height = info.text.strip()

    # Extracting tables for per game stats and advanced stats
    tables = soup.find_all('table')
    stats = {}
    for table in tables:
        if table.get('id') in ['players_per_game', 'players_advanced']:
            df = pd.read_html(str(table))[0]
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(0)  # Dropping multi-level column headers only if they exist
            stats[table.get('id')] = df
            
    if stats != {}:
        try:
            players_per_game = stats.get('players_per_game')
            players_advanced = stats.get('players_advanced')

            pg_cols = ["G", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "SOS"]
            adv_cols = ["PER", "TS%", "eFG%", "3PAr", "FTr", "PProd", "ORB%", "DRB%", "TRB%", "AST%", "STL%", "BLK%", "TOV%", "USG%", "OWS", "DWS", "OBPM", "DBPM"]

            players_per_game = players_per_game.loc[:,pg_cols]
            players_advanced = players_advanced.loc[:,adv_cols]
            return height, players_per_game, players_advanced
        except:
            pass
    return height, None, None

def get_player_information(first, last):
    """
    Obtains players date of birth from basketballreference.com.
    """
    url = f'https://www.basketball-reference.com/players/{last[0].lower()}/{last[:5].lower() + first[:2].lower()}01.html'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    element_with_date_birth = soup.find('span', attrs={'data-birth': True})

    # Extract the 'data-birth' attribute
    date_birth = element_with_date_birth['data-birth'] if element_with_date_birth else None

    return date_birth

def get_draft_class(year):
    """
    Returns a list of player names from the given NBA draft year.
    """
    url = f"https://www.basketball-reference.com/draft/NBA_{year}.html"
    response = requests.get(url)
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the player names
    # Assuming the player names are in 'td' tags with 'data-stat' attribute set to 'player'
    player_name_cells = soup.find_all('td', {'data-stat': 'player'})

    # Extract and print the player names
    player_names = [cell.get_text() for cell in player_name_cells]
    return player_names

In [ ]:
# Replace the URL with the page you want to scrape
height, stats = get_player_stats("zion", "williamson")
birth_date = get_player_information("Zion", "Williamson")

players_per_game = stats.get('players_per_game')
players_advanced = stats.get('players_advanced')

players_per_game.drop(columns=["Season", "School", "Conf", "Class", "GS", "2P", "2PA", "2P%", "ORB", "DRB", "Unnamed: 28"], inplace=True)
players_advanced.drop(columns=["Season", "School", "Conf", "Class", "G", "GS", "MP", "Unnamed: 21", "Unnamed: 26", "BPM", "WS", "WS/40"], inplace=True)

print("Height and Weight:", height)
print("Birth Date:", birth_date)
print("\nPer Game Stats:\n", players_per_game.iloc[-2])
print("\nAdvanced Stats:\n", players_advanced.iloc[-2])

# SQL functions

In [9]:
def sql_connection():
    """
    Creates a connection to the postgresql server
    """
    
    load_dotenv()

    # Accessing variables from .env file
    db_name = os.getenv('DB_NAME')
    db_user = os.getenv('DB_USER')
    db_pass = os.getenv('DB_PASS')
    db_host = os.getenv('DB_HOST')
    db_port = os.getenv('DB_PORT')
    
    try:
        conn = psycopg2.connect(
            dbname=db_name,
            user=db_user,
            password=db_pass,
            host=db_host,
            port=db_port
        )
        print("Connected to the database.")
        return (conn, conn.cursor())
    except Exception as e:
        print("Unable to connect to the database.")
        print(e)
        return None

def insert_player(conn, cursor, player_tuple):
    """
    Inserts a row into the Player table
    
    cursor : SQL connection cursor
    player_tuple : <tuple> contains player information
    
    Returns the created ID of the player
    """
    insert_statement = """
    INSERT INTO "Players" ("PlayerName", "DOB", "Rating", "DraftYear") VALUES (%s, %s, %s, %s) RETURNING "PlayerID";
    """
    try:
        cursor.execute(insert_statement, player_tuple)
        generated_id = cursor.fetchone()[0]  # Fetch the generated ID
        conn.commit()
        print(player_tuple[0])
        print("Added Player Successfully")
        return generated_id
    except Exception as e:
        print("Unable to insert to the Players Table.")
        print(e)
        return None
    
def insert_stats(conn, cursor, stat_tuple):
    """
    Inserts a row into the PlayerStats table
    
    conn : SQL connection
    cursor : SQL connection cursor
    stat_dict : <tuple> contains stat to be insert into the table
    """
    insert_statement = """
    INSERT INTO "PlayerStats" ("PlayerID", "Games", "MPG", "FG", "FGA", "FGP", "3P", "3PA", "3PP", "FT", "FTA", "FTP", "RPG", "APG", "SPG", "BPG", "TOV", "PF", "PPG", "SOS", "PER", "TS", "eFG", "3PAr", "FTr", "PProd", "ORBP", "DRBP", "TRBP", "ASTP", "STLP", "BLKP", "TOVP", "USG", "OWS", "DWS", "OBPM", "DBPM") VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING "StatID";
    """
    
    try:
        cursor.execute(insert_statement, stat_tuple)
        generated_id = cursor.fetchone()[0]  # Fetch the generated ID
        conn.commit()
        print("Added Stat Successfully")
        return generated_id
    except Exception as e:
        print("Unable to insert to the Stats Table.")
        print(e)
        print("Data tuple:", stat_tuple)
        return None

def get_all_players(cursor):
    """
    Returns all players in the database
    """
    query = 'SELECT * FROM "Players"'
    cursor.execute(query)
    records = cursor.fetchall()
    for record in records:
        print(record)

def get_all_stats(cursor):
    """
    Returns all players in the database
    """
    query = 'SELECT * FROM "PlayerStats"'
    cursor.execute(query)
    records = cursor.fetchall()
    for record in records:
        print(record)

In [18]:
get_all_players(cursor)

(2, 'Anthony Bennett', datetime.date(1993, 3, 14), 0, 2013)
(3, 'Victor Oladipo', datetime.date(1992, 5, 4), 0, 2013)
(4, 'Cody Zeller', datetime.date(1992, 10, 5), 0, 2013)
(5, 'Alex Len', datetime.date(1993, 6, 16), 0, 2013)
(6, 'Nerlens Noel', datetime.date(1994, 4, 10), 0, 2013)
(7, 'Ben McLemore', datetime.date(1993, 2, 11), 0, 2013)
(8, 'Kentavious Caldwell-Pope', datetime.date(1993, 2, 18), 0, 2013)
(9, 'Trey Burke', datetime.date(1992, 11, 12), 0, 2013)
(10, 'CJ McCollum', datetime.date(1991, 9, 19), 0, 2013)
(11, 'Michael Carter-Williams', datetime.date(1991, 10, 10), 0, 2013)
(12, 'Steven Adams', datetime.date(1993, 7, 20), 0, 2013)
(13, 'Kelly Olynyk', datetime.date(1991, 4, 19), 0, 2013)
(14, 'Shabazz Muhammad', datetime.date(1992, 11, 13), 0, 2013)
(15, 'Shane Larkin', datetime.date(1992, 10, 2), 0, 2013)
(16, 'Tony Snell', datetime.date(1991, 11, 10), 0, 2013)
(17, 'Gorgui Dieng', datetime.date(1990, 1, 18), 0, 2013)
(18, 'Mason Plumlee', datetime.date(1990, 3, 5), 0, 201

In [17]:
conn, cursor = sql_connection()

Connected to the database.


In [14]:
def find_names(fullname):
    """
    Takes in a full name and returns a tuple of the first and last name in lower case
    """
    space = fullname.index(" ")
    first = fullname[:space].lower()
    last = fullname[space+1:].lower()
    return (first, last)

def fill_draft_class(conn, cursor, year):
    """
    Fills the database with players from the given draft year
    """
    inserted = []
    player_names = get_draft_class(year)
    for player in player_names:
        print("Attempting to add " + player)
        
        first, last = find_names(player)
        height, players_per_game, players_advanced = get_player_stats(first, last)
        time.sleep(5)
        birth_date = get_player_information(first, last)
        time.sleep(5)
        
        if height is not None and players_per_game is not None and players_advanced is not None and birth_date is not None:
            player_tuple = (player, birth_date, 0, year)
            playerID = insert_player(conn, cursor, player_tuple)
            
            if playerID is not None:
                
                stat_tuple = tuple([playerID] + players_per_game.iloc[-2].tolist() + players_advanced.iloc[-2].tolist())
                insert_stats(conn, cursor, stat_tuple)
                
                inserted.append(player)
                
    return inserted

def fill_drafts(conn, cursor, start, end):
    """
    Fills the database with information on players from draft classes start-end
    """
    players = []
    for draft_year in range(start, end+1):
        print(draft_year)
        draft_players = fill_draft_class(conn, cursor, draft_year)
        players += draft_players
        time.sleep(5)
    
    print("end")
    conn.close()
    return players


In [15]:
names = fill_drafts(conn, cursor, 2017, 2019)

2017
Attempting to add Markelle Fultz
Markelle Fultz
Added Player Successfully
Added Stat Successfully
Attempting to add Lonzo Ball
Lonzo Ball
Added Player Successfully
Added Stat Successfully
Attempting to add Jayson Tatum
Jayson Tatum
Added Player Successfully
Added Stat Successfully
Attempting to add Josh Jackson
Attempting to add De'Aaron Fox
Attempting to add Jonathan Isaac
Jonathan Isaac
Added Player Successfully
Added Stat Successfully
Attempting to add Lauri Markkanen
Lauri Markkanen
Added Player Successfully
Added Stat Successfully
Attempting to add Frank Ntilikina
Attempting to add Dennis Smith Jr.
Attempting to add Zach Collins
Zach Collins
Added Player Successfully
Added Stat Successfully
Attempting to add Malik Monk
Malik Monk
Added Player Successfully
Added Stat Successfully
Attempting to add Luke Kennard
Luke Kennard
Added Player Successfully
Added Stat Successfully
Attempting to add Donovan Mitchell
Donovan Mitchell
Added Player Successfully
Added Stat Successfully
Atte

Thomas Welsh
Added Player Successfully
Added Stat Successfully
Attempting to add George King
George King
Added Player Successfully
Added Stat Successfully
Attempting to add Kostas Antetokounmpo
Kostas Antetokounmpo
Added Player Successfully
Added Stat Successfully
2019
Attempting to add Zion Williamson
Zion Williamson
Added Player Successfully
Added Stat Successfully
Attempting to add Ja Morant
Attempting to add RJ Barrett
RJ Barrett
Added Player Successfully
Added Stat Successfully
Attempting to add De'Andre Hunter
Attempting to add Darius Garland
Darius Garland
Added Player Successfully
Added Stat Successfully
Attempting to add Jarrett Culver
Jarrett Culver
Added Player Successfully
Added Stat Successfully
Attempting to add Coby White
Coby White
Added Player Successfully
Added Stat Successfully
Attempting to add Jaxson Hayes
Jaxson Hayes
Added Player Successfully
Added Stat Successfully
Attempting to add Rui Hachimura
Rui Hachimura
Added Player Successfully
Added Stat Successfully
At

In [19]:
query = 'SELECT "PlayerID", "PlayerName" FROM "Players";'
cursor.execute(query)
records = cursor.fetchall()
print(records)

[(2, 'Anthony Bennett'), (3, 'Victor Oladipo'), (4, 'Cody Zeller'), (5, 'Alex Len'), (6, 'Nerlens Noel'), (7, 'Ben McLemore'), (8, 'Kentavious Caldwell-Pope'), (9, 'Trey Burke'), (10, 'CJ McCollum'), (11, 'Michael Carter-Williams'), (12, 'Steven Adams'), (13, 'Kelly Olynyk'), (14, 'Shabazz Muhammad'), (15, 'Shane Larkin'), (16, 'Tony Snell'), (17, 'Gorgui Dieng'), (18, 'Mason Plumlee'), (19, 'Solomon Hill'), (20, 'Andre Roberson'), (21, 'Archie Goodwin'), (22, 'Allen Crabbe'), (23, 'Carrick Felix'), (24, 'Isaiah Canaan'), (25, 'Nate Wolters'), (26, 'Jeff Withey'), (27, 'Grant Jerrett'), (28, 'Jamaal Franklin'), (29, 'Pierre Jackson'), (30, 'Mike Muscala'), (31, 'Erick Green'), (32, 'Erik Murphy'), (33, 'Romero Osby'), (34, 'Lorenzo Brown'), (35, 'Colton Iverson'), (36, 'Arsalan Kazemi'), (37, 'Peyton Siva'), (38, 'Alex Oriakhi'), (39, 'Deshaun Thomas'), (40, 'Andrew Wiggins'), (41, 'Jabari Parker'), (42, 'Joel Embiid'), (43, 'Aaron Gordon'), (44, 'Marcus Smart'), (45, 'Julius Randle'),

In [20]:
import csv
with open('output.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write each item as a new row
    for pid, name in records:
        writer.writerow([pid, name])

In [ ]:
get_draft_class(2013)